# Linear Regression with Stellar Spectra

- By Yuan-Sen Ting, January 2025

In this tutorial, we'll apply linear regression to analyze stellar spectra from the Sloan Digital Sky Survey's Apache Point Observatory Galactic Evolution Experiment (SDSS APOGEE). Our goal is to predict stellar temperatures from spectral data, illustrating how linear regression can extract physical information from astronomical observations.

## Prerequisites

### What You Should Already Know

Before starting this lab, you should be familiar with:

1. **Linear Regression & Maximum Likelihood**: You should understand how linear regression works through maximum likelihood estimation (MLE), as covered in the lecture.
2. **NumPy Matrix Operations**: You'll need to be comfortable with matrices in NumPy for data manipulation.
3. **Basic Statistics**: Understanding of concepts like mean, variance, and probability distributions.
4. **Bayesian Concepts**: Basic familiarity with Bayesian statistics and prior distributions.

### What You'll Learn

After completing this lab, you'll be able to:

1. **Apply Linear Regression to Stellar Data**: Learn how to use linear regression for analyzing APOGEE spectra.
2. **Implement Maximum Likelihood**: Use MLE to find optimal model parameters.
3. **Apply Regularization**: Use regularization techniques to prevent overfitting.

Let's begin by setting up our Python environment with the necessary libraries.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

# Set higher DPI for better looking plots
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# For reproducibility 
np.random.seed(42)

## The Dataset
  
We'll be working with 6,500 stellar spectra from APOGEE (Apache Point Observatory Galactic Evolution Experiment). APOGEE has been mapping out stars across the Milky Way, providing us with detailed spectroscopic data about their properties.
  
### Key Variables
  
1. **Spectra (`spectrum_array`)**: Raw spectral measurements for our 6,500 stars. Each spectrum has 7214 data points.

2. **Wavelength (`wavelength`)**: The wavelength values corresponding to our spectral data points, helping us identify specific absorption lines.
  
3. **Effective Temperature (`teff_array`)**: Surface temperatures of the stars - this is what we'll try to predict with our regression model.
    
4. **Surface Gravity (`logg_array`)**: Log of surface gravity (in cgs units) for each star. We won't use this in today's analysis.
  
5. **Metallicity (`feh_array`)**: Measures how metal-rich each star is compared to the Sun. Also not used in this tutorial.
   
### Data Quality
   
For this tutorial, we're using high-quality spectra with mostly signal-to-noise ratios (SNR) > 50. While there are some minor measurement uncertainties of the spectra in our dataset, we'll assume the input is noiseless. This is because in our lecture, we only covered the case where there is noise in the output (target) variable, but not in the input features. In real astronomical data analysis, properly accounting for measurement uncertainties in both inputs and outputs often requires more sophisticated techniques.  Which we will cover in the Lecture 6. For now, we'll use the simpler approach to demonstrate the core concepts.
  
Let's load our data and take a look at what we're working with.

In [2]:
# Read data from the file
file = np.load('dataset_apogee_spectra.npz')

# Extract spectral data and uncertainties
spectra = file['spectrum_array']
wavelength = file['wavelength']

# Extract stellar parameters
teff_array = file['teff_array']
logg_array = file['logg_array']
feh_array = file['feh_array']

# Print basic information about our dataset
print(f"Number of stars: {len(spectra)}")
print(f"Points per spectrum: {len(wavelength)}")
print(f"Temperature range: {np.min(teff_array):.0f}K - {np.max(teff_array):.0f}K")
print(f"\nTypical uncertainty range in spectra:")

## Visualizing a Sample Spectrum
 
Let's examine what our data actually looks like by plotting a spectrum. 


In [3]:
# Plot a portion of the spectrum for the first star with uncertainties
# We'll take only the first 1000 points for clearer visualization
plt.figure(figsize=(8, 4))

# Plot the spectrum
plt.errorbar(wavelength[:1000], spectra[0, :1000],
             alpha=0.5, ecolor='gray', capsize=0)

# Add labels and title
plt.xlabel(r'Wavelength ($\mathrm{\AA}$)')
plt.ylabel('Normalized Flux')
plt.title('Sample APOGEE Spectrum with Uncertainties')

# Add a text box with stellar parameters
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
text = fr'$T_{{\mathrm{{eff}}}}$ = {teff_array[0]:.0f} K' + '\n' + \
       fr'$\log g$ = {logg_array[0]:.2f}' + '\n' + \
       fr'$[\mathrm{{Fe/H}}]$ = {feh_array[0]:.2f}'
plt.text(0.05, 0.2, text, transform=plt.gca().transAxes, 
         verticalalignment='top', bbox=props)

plt.show()


## Exploring Our Sample: The Kiel Diagram

Let's look at the Kiel diagram - it's a useful way to visualize stellar properties by plotting surface gravity against effective temperature. We'll also color-code the points by metallicity to add another dimension of information.
 
Our APOGEE data doesn't have many dwarf stars. There are a few reasons for this:
 
1. **APOGEE's Design**: The survey was built to study giant stars - its pipeline works best with them.
   
2. **Data Quality**: We're only using high signal-to-noise data here. Since dwarf stars are typically fainter, their spectra often don't make this quality cut.

This visualization will help us understand the parameter space covered by our training data, which is important for understanding where our regression model will be most reliable.

In [4]:
# Create a scatter plot of Surface Gravity vs Effective Temperature, color-coded by Metallicity
plt.figure(figsize=(8, 5))
scatter = plt.scatter(teff_array, logg_array, c=feh_array, 
                     cmap='viridis', s=10, alpha=0.6)

# Label axes and add title
plt.xlabel('Effective Temperature [K]')
plt.ylabel('Surface Gravity [log g]')
plt.title('Kiel Diagram of APOGEE Sample')

# Add a colorbar
plt.colorbar(scatter, label='[Fe/H]')

# Invert both axes for traditional presentation
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

# Print some basic statistics about our sample
print("Sample Statistics:")
print(f"Temperature range: {np.min(teff_array):.0f}K - {np.max(teff_array):.0f}K")
print(f"Surface gravity range: {np.min(logg_array):.2f} - {np.max(logg_array):.2f}")
print(f"Metallicity range: {np.min(feh_array):.2f} - {np.max(feh_array):.2f}")

## Linear Regression: Starting Simple
  
Let's start with basic linear regression to predict effective temperature from stellar spectra. We'll use the spectra as our input features ($\mathbf{x}$) and temperature as our target variable ($y$).
  
In linear regression, we often use the symbol $\mathbf{\Phi}$ (Phi) to represent our design matrix or feature matrix. While $\mathbf{x}$ represents raw input features, $\mathbf{\Phi}$ typically denotes the features after any transformations or preprocessing. In our case, $\mathbf{\Phi}$ will contain our spectral data plus a bias term.
  
### Initial Simplifications
  
For our first attempt, we'll make a few simplifying assumptions:
 
1. **Ignore Uncertainties**: We'll initially treat all measurements as equally reliable, ignoring the uncertainty information we have.
2. **Limited Features**: Instead of using all 7214 wavelength points, we'll start with just the first 2000 pixels to keep things computationally manageable.
3. **Add Bias Term**: We'll add a bias term to our model by augmenting our feature matrix $\mathbf{\Phi}$ with a column of ones.
 
These simplifications will help us establish a baseline model before adding more sophisticated treatments of regularization and feature engineering.
 
Let's prepare our data for regression:

In [5]:
# Use spectra as the design matrix and limit to first 2000 pixels
Phi = spectra[:,:2000]

# Add a column of ones to the design matrix to act as the bias term
Phi = np.hstack((Phi, np.ones((Phi.shape[0], 1))))
print("Shape of design matrix Phi:", Phi.shape)

# Use effective temperature as the target variable
t = teff_array
print("Shape of target array t:", t.shape)

# Split data into training and test sets
from sklearn.model_selection import train_test_split
Phi_train, Phi_test, t_train, t_test = train_test_split(Phi, t, test_size=0.2, random_state=42)
print("\nTraining set size:", Phi_train.shape[0])
print("Test set size:", Phi_test.shape[0])

## Maximum Likelihood Estimation in Linear Regression

As we saw in the lecture, for linear regression with homogeneous noise (constant variance), the maximum likelihood estimate has an analytical solution:

$$\mathbf{w}_{\text{ML}} = (\mathbf{\Phi}_{\text{train}}^T \mathbf{\Phi}_{\text{train}})^{-1} \mathbf{\Phi}_{\text{train}}^T \mathbf{t}_{\text{train}}$$

This solution emerges from maximizing the likelihood function under the assumption of Gaussian noise with constant variance. It provides a good starting point for our analysis.

Let's implement this solution and examine how well it predicts stellar temperatures:

In [6]:
# Compute the Maximum Likelihood weights using the analytical solution
w_ml = np.linalg.inv(Phi_train.T @ Phi_train) @ Phi_train.T @ t_train

# Make predictions on both training and test sets
t_pred_train = Phi_train @ w_ml
t_pred_test = Phi_test @ w_ml

# Calculate RMSE for both sets
train_RMSE = np.sqrt(np.mean((t_train - t_pred_train)**2))
test_RMSE = np.sqrt(np.mean((t_test - t_pred_test)**2))

print("Training RMSE: {:.2f}K".format(train_RMSE))
print("Test RMSE: {:.2f}K".format(test_RMSE))

# Create scatter plot of predicted vs actual temperatures
plt.figure(figsize=(10, 4))

# Training set
plt.subplot(1, 2, 1)
plt.scatter(t_train, t_pred_train, alpha=0.5, s=1)
plt.plot([t_train.min(), t_train.max()], [t_train.min(), t_train.max()], 
         'r--', label='Perfect Prediction')
plt.xlabel('True Temperature [K]')
plt.ylabel('Predicted Temperature [K]')
plt.title('Training Set')
plt.legend()

# Test set
plt.subplot(1, 2, 2)
plt.scatter(t_test, t_pred_test, alpha=0.5, s=1)
plt.plot([t_test.min(), t_test.max()], [t_test.min(), t_test.max()], 
         'r--', label='Perfect Prediction')
plt.xlabel('True Temperature [K]')
plt.ylabel('Predicted Temperature [K]')
plt.title('Test Set')
plt.legend()

plt.tight_layout()
plt.show()


### Analyzing Our First Results

Our simple linear regression model is performing reasonably well, achieving a precision of around 130K in temperature predictions. To put this in context:

1. **Data Usage**: We're currently only using the first 2000 pixels of each spectrum (less than 1/3 of the available data) and treating all measurements as equally reliable.

2. **Benchmark Performance**: Major spectroscopic surveys like APOGEE typically achieve temperature uncertainties of around 50-100K. While our simple model isn't quite at that level, it's encouraging that we're in the right ballpark considering how basic our approach is.

3. **Room for Improvement**: Several factors could potentially improve our predictions:
   - Including more spectral features
   - Using regularization to prevent overfitting
   - Engineering better features based on our knowledge of stellar physics


## Exploring the Effect of Regularization

As discussed in the lecture, when building machine learning models, we often encounter overfitting - where our model gets too good at predicting the training data but fails to generalize well to new data. Regularization helps prevent this by adding a penalty term that discourages complex solutions.

From the lecture, we saw that adding a zero-mean Gaussian prior with variance $\eta^2$ leads to the regularized solution:

$$\mathbf{w}_{\text{ML}} = (\lambda \mathbf{I} + \mathbf{\Phi}^T\mathbf{\Phi})^{-1}\mathbf{\Phi}^T\mathbf{t}$$

where $\lambda = \sigma^2/\eta^2$ is our regularization parameter. This parameter controls the trade-off between:
- Fitting the data well (small $\lambda$)
- Keeping the weights small to prevent overfitting (large $\lambda$)

Let's implement this regularized solution and explore how different values of $\lambda$ affect our predictions:

In [7]:
# Define a function to calculate regularized weights
def w_ml_regularised(l):
    return np.linalg.inv(l * np.eye(Phi_train.shape[1]) + Phi_train.T @ Phi_train) @ Phi_train.T @ t_train

# Define a range of log-scaled lambda values
log_lambdas = np.linspace(-2, 2, 16)

# Initialize array to store results
# First column: training RMSE, Second column: test RMSE
results = np.zeros((len(log_lambdas), 2))

# Loop over each hyperparameter value and train the model
for ix, l in enumerate(log_lambdas):
    # Compute the regularized weight vector
    w_reg = w_ml_regularised(10**float(l))
    
    # Calculate RMSE for training and test sets
    rmse_train = np.sqrt(np.mean((t_train - Phi_train @ w_reg)**2))
    rmse_test = np.sqrt(np.mean((t_test - Phi_test @ w_reg)**2))
    
    # Store the results
    results[ix, 0] = rmse_train
    results[ix, 1] = rmse_test
    
# Plot the RMSE values as a function of lambda
plt.figure(figsize=(8, 4))
plt.plot(log_lambdas, results[:,0], 'b-', label='Train')
plt.plot(log_lambdas, results[:,1], 'r-', label='Test')
plt.xlabel(r'log($\lambda$)')
plt.ylabel('RMSE [K]')
plt.title('RMSE vs Regularization Parameter')
plt.legend()
plt.grid(True)
plt.show()

# Find best lambda and its performance
best_lambda = 10**log_lambdas[np.argmin(results[:,1])]
print(f"Best lambda: {best_lambda:.2f}")
print(f"Best test RMSE: {np.min(results[:,1]):.2f}K")

# Compute final predictions using best lambda
w_best = w_ml_regularised(best_lambda)
t_pred_test_reg = Phi_test @ w_best

# Plot comparison of predictions
plt.figure(figsize=(8, 4))
plt.scatter(t_test, t_pred_test_reg, alpha=0.5, s=1)
plt.plot([t_test.min(), t_test.max()], [t_test.min(), t_test.max()], 
         'r--', label='Perfect Prediction')
plt.xlabel('True Temperature [K]')
plt.ylabel('Predicted Temperature [K]')
plt.title('Predictions with Optimal Regularization')
plt.legend()
plt.show()

# Print comparison of all methods
print("\nRMSE Comparison:")
print(f"Without Regularization: {test_RMSE:.2f}K")
print(f"Regularized: {np.min(results[:,1]):.2f}K")

### Impact of Regularization

The regularization results show a meaningful improvement in our model's performance:

1. **Significant Improvement**: With optimal regularization, we achieve an RMSE of ~115K on the test set, a substantial improvement from our previous RMSE of ~130K. This ~10% improvement in prediction accuracy demonstrates the value of regularization in our spectral analysis.

2. **Training vs Test Performance**: The plot of RMSE against λ shows the classic bias-variance trade-off:
   - At very low λ values, we see overfitting (low training error but high test error)
   - At very high λ values, we see underfitting (both training and test error increase)
   - The optimal λ provides the best balance between these extremes

3. **Model Comparison**:
   - Basic Linear Regression: ~130K RMSE
   - With Optimal Regularization: ~115K RMSE
   - For context, APOGEE achieves uncertainties of around 50-100K in their pipeline

This improvement suggests that our initial models were indeed overfitting the data, and that regularization helps by constraining the model weights to focus on the most relevant spectral features for temperature determination.

In our next section, we'll explore whether we can further improve performance through feature engineering - transforming our input features based on our knowledge of stellar physics.

## Extending Linear Models with Basis Functions

As discussed in the lecture, linear regression becomes more powerful when we transform our input features using basis functions. These transformations allow us to capture nonlinear relationships while maintaining the computational advantages of linear regression.

For our spectral analysis, we'll explore quadratic features. 

Let's implement a quadratic basis function that takes each feature $\mathbf{x}$ and creates two new features: $[\mathbf{x}, \mathbf{x}^2]$. This allows our model to capture both linear and quadratic relationships between spectral features and temperature.

In [8]:
# Function to create quadratic features
def phi_quadratic(x):    
    return np.hstack((x, x**2))

# Extend the feature set
Phi_quad_train = np.array([phi_quadratic(Phi_train[i]) for i in range(Phi_train.shape[0])])
Phi_quad_test = np.array([phi_quadratic(Phi_test[i]) for i in range(Phi_test.shape[0])])

# Remove the extra bias term that got duplicated
Phi_quad_train = Phi_quad_train[:,:-1]
Phi_quad_test = Phi_quad_test[:,:-1]

# First try without regularization
w_unreg = np.linalg.inv(Phi_quad_train.T @ Phi_quad_train) @ Phi_quad_train.T @ t_train
t_train_pred = Phi_quad_train @ w_unreg
t_test_pred = Phi_quad_test @ w_unreg

# Calculate RMSE
train_rmse = np.sqrt(np.mean((t_train - t_train_pred)**2))
test_rmse = np.sqrt(np.mean((t_test - t_test_pred)**2))

print("RMSE with quadratic features (no regularization):")
print(f"Train: {train_rmse:.2f}K")
print(f"Test: {test_rmse:.2f}K")

# Now try with regularization
def w_ml_regularised_quad(l):
    return np.linalg.inv(l * np.eye(Phi_quad_train.shape[1]) + 
                        Phi_quad_train.T @ Phi_quad_train) @ Phi_quad_train.T @ t_train

# Search over regularization parameters
log_lambdas = np.linspace(-2, 2, 16)
results_quad = np.zeros((len(log_lambdas), 2))

for ix, l in enumerate(log_lambdas):
    w_reg = w_ml_regularised_quad(10**float(l))
    rmse_train = np.sqrt(np.mean((t_train - Phi_quad_train @ w_reg)**2))
    rmse_test = np.sqrt(np.mean((t_test - Phi_quad_test @ w_reg)**2))
    results_quad[ix, 0] = rmse_train
    results_quad[ix, 1] = rmse_test

# Plot results
plt.figure(figsize=(8, 4))
plt.plot(log_lambdas, results_quad[:,0], 'b-', label='Train')
plt.plot(log_lambdas, results_quad[:,1], 'r-', label='Test')
plt.xlabel(r'log($\lambda$)')
plt.ylabel('RMSE [K]')
plt.title('RMSE vs Regularization Parameter (Quadratic Features)')
plt.legend()
plt.grid(True)
plt.show()

# Find best lambda and its performance
best_lambda_quad = 10**log_lambdas[np.argmin(results_quad[:,1])]
print(f"\nBest lambda: {best_lambda_quad:.2f}")
print(f"Best test RMSE with quadratic features: {np.min(results_quad[:,1]):.2f}K")

# Print comparison of all methods
print("\nFinal RMSE Comparison:")
print(f"Linear (no regularization): {test_RMSE:.2f}K")
print(f"Linear (with regularization): {np.min(results[:,1]):.2f}K")
print(f"Quadratic (with regularization): {np.min(results_quad[:,1]):.2f}K")

### Analysis of Feature Engineering Results

Our experiments with quadratic features reveal some important lessons about model complexity and overfitting:

1. **Severe Overfitting Without Regularization**: 
   - Training RMSE: 17K
   - Test RMSE: 225K
   This dramatic difference between training and test performance is a textbook example of overfitting. Our model has learned to fit the training data almost perfectly (17K precision!) but fails catastrophically on new data.

2. **Recovery with Regularization**: Adding regularization helps control this overfitting:
   - Linear (no regularization): ~130K
   - Linear (with regularization): ~110K
   - Quadratic (with regularization): ~110K

   While regularization helps recover reasonable performance, the quadratic features only provide a very modest improvement over the regularized linear model.

3. **Insights**:
   - Adding complexity (quadratic terms) without proper constraints (regularization) can dramatically hurt generalization
   - Even with regularization, doubling our number of features only yields marginal improvements
   - The simpler regularized linear model might be preferable given the minimal gains from added complexity

This exercise perfectly illustrates why we need both:
- Feature engineering to potentially capture more complex relationships
- Regularization to prevent our more complex models from overfitting

It also demonstrates that more complex models aren't always better - the modest improvement from quadratic features might not justify the increased computational cost and model complexity in practice.

## Conclusion
 
In this tutorial, we've explored linear regression through the lens of stellar spectroscopy, progressing from simple models to more sophisticated approaches. Let's summarize our key findings:

### Model Evolution and Performance
1. **Basic Linear Regression**: ~130K RMSE
   - Simple to implement
   - Reasonable baseline performance
   - No protection against overfitting

2. **Regularized Linear Model**: ~115K RMSE
   - Significant improvement over baseline
   - Prevents overfitting through controlled parameter constraints
   - Good balance of complexity and performance

3. **Quadratic Features**: 
   - Without regularization: Severe overfitting (17K train / 225K test RMSE)
   - With regularization: ~110K RMSE
   - Minimal improvement over linear model despite doubled complexity

### Key Lessons Learned

1. **The Power of Simplicity**
   - Simple models with proper regularization can outperform more complex unregularized models
   - Adding complexity needs to be justified by meaningful performance improvements

2. **Importance of Validation**
   - Training performance alone can be misleading (as seen in quadratic features)
   - Need to check generalization performance on test set
   - Large gaps between training and test performance signal overfitting

3. **The Role of Domain Knowledge**
   - Understanding your data (like homogeneous uncertainties in APOGEE)
   - Realistic expectations for model performance (comparing to APOGEE pipeline's ~50-100K)
   - Making informed choices about model complexity

### Future Directions

Potential ways to further improve this model could include:
- Using more of the available spectral features (we only used first 2000 points)
- Incorporating physical knowledge about specific temperature-sensitive spectral regions
- Exploring other basis functions based on stellar physics
- Implementing a full Bayesian treatment to get uncertainty estimates in our predictions

This tutorial demonstrates how principles from the lecture - like maximum likelihood estimation, regularization, and feature engineering - apply to real astronomical data analysis challenges.